In [98]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=16, app_name='west0_mclaren', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 16 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Still waiting, hang tight plz! elapsed: 00:01:00
Still waiting, hang tight plz! elapsed: 00:02:01
Spark master launched!
Creating new Spark session, name: west0_mclaren...
Waiting for all executors ready...
7/16 executors ready
8/16 executors ready
10/16 executors ready
11/16 executors ready
14/16 executors ready
15/16 executors ready
All executors connected!
Complete! elapsed time: 00:03:28


In [97]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [109]:
df = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info_tmp")

In [57]:
# mysql.insert_table(df, "labs", "mclaren_seg_pre_and_post_info_tmp")

In [66]:
# df = df.na.fill(0)

In [68]:
# mysql.insert_table(df, "labs", "mclaren_seg_pre_and_post_info")

In [29]:
tm_user = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info").where(col("mclaren_group") == "TM")
mclaren_owner = mysql.read_table(spark, "labs", "mclaren_owner")

In [33]:
mclaren_owner.withColumnRenamed("AccountId", "account_id").join(tm_user.select("account_id"), "account_id").groupBy("account_id").agg(concat_ws(",", sort_array(collect_list(col("mclaren")))).alias("mclarens")) \
    .groupBy("mclarens").agg(count("*").alias("user_cnt")).orderBy("user_cnt", ascending=False).toPandas()

,mclarens,user_cnt
0,"elite_yellow,standard_black",25102
1,"standard_black,standard_white",7608
2,"elite_yellow,standard_black,standard_white",872
3,"elite_yellow,standard_white",359


In [16]:
from datetime import datetime, timedelta

def load_pass(startdate, enddate):
    df = None
    startdate = datetime.strptime(startdate, '%Y-%m-%d')
    enddate = datetime.strptime(enddate, '%Y-%m-%d')
    while startdate <= enddate :
        startdate = startdate.strftime('%Y-%m-%d')
        if df is None:
            df = spark.read.parquet("s3a://pubg-log-labs/data_mart/pass_master/pc/"+startdate)
        else :
            df = df.union(spark.read.parquet("s3a://pubg-log-labs/data_mart/pass_master/pc/"+startdate))
        startdate = datetime.strptime(startdate, '%Y-%m-%d') + timedelta(days = 1)
    return df

In [17]:
post_pass_end_date = "2022-11-01"
pass_master = load_pass(post_pass_end_date, post_pass_end_date).where(col("period_type") == "mclaren")

In [18]:
pass_user = pass_master.where(col("period_status") == "pass")

In [19]:
m_user = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info").where(col("group").isin(["M-1", "M-2"]))

In [25]:
pass_but_m_user = pass_user.select("AccountId").withColumnRenamed("AccountId", "account_id").join(m_user, "account_id")

In [23]:
pass_but_m_user.groupBy("group", "mclaren_group").agg(countDistinct("account_id")).show(truncate=False)

+-----+-------------+--------------------------+
|group|mclaren_group|count(DISTINCT account_id)|
+-----+-------------+--------------------------+
|M-1  |N            |49                        |
|M-2  |TM           |1                         |
|M-2  |SW           |6                         |
|M-2  |SB           |5                         |
|M-2  |EY           |2                         |
|M-2  |N            |43                        |
+-----+-------------+--------------------------+



In [27]:
mysql.insert_table(pass_but_m_user.select("account_id", "group", "mclaren_group"), "labs", "pass_but_m_user")

In [99]:
special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", "2022-09-07", "2022-11-08").where(col("Currency").like("%seasonaltoken%"))

In [4]:
seasonal_token = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-09-07", "2022-11-08").where(col("Currency").like("%seasonaltoken%"))

In [4]:
special_craft = special_craft.groupBy("AccountId", "ItemId").agg(sum("Amount").alias("cnt"))

In [8]:
special_craft.select(countDistinct("ItemId")).show()

+----------------------+
|count(DISTINCT ItemId)|
+----------------------+
|                    24|
+----------------------+



In [3]:
items = mysql.read_table(spark, "metainfo", "meta_vc_sales_items").where(col("event_name") == "202209_season_workshop").select("product_id", "product_name").distinct()

In [5]:
special_craft = special_craft.withColumnRenamed("AccountId", "account_id").withColumnRenamed("ItemId", "product_id").join(items, "product_id", "left")

In [6]:
special_craft_without_mclaren = special_craft.where(~col("product_id").isin(["itemdesc.12021003", "itemdesc.12021001"]))

In [38]:
df = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info")

In [39]:
user_group_info = df.select("account_id", "group", "mclaren_group")

In [9]:
mclaren_user_special_craft = user_group_info.join(special_craft_without_mclaren, "account_id", "left")

In [12]:
mclaren_user_special_craft = mclaren_user_special_craft.withColumn("special_craft_user", when(col("product_id").isNull(), lit(None)).otherwise(col("account_id")))

In [44]:
mclaren_user_special_craft.groupBy("group", "mclaren_group").agg(countDistinct("account_id").alias("user_cnt_in_group"), countDistinct("special_craft_user").alias("special_craft_user_cnt_in_group")).orderBy("group", "mclaren_group").toPandas()

,group,mclaren_group,user_cnt_in_group,special_craft_user_cnt_in_group
0,M-1,N,3153593,95689
1,M-1,SB,13467,5091
2,M-2,EY,377,265
3,M-2,N,190355,32938
4,M-2,SB,10252,8191
5,M-2,SW,1067,656
6,M-2,TM,614,486
7,P-1,EY,71,35
8,P-1,N,71176,26665
9,P-1,SB,4761,4530


In [50]:
mclaren_user_special_craft

DataFrame[account_id: string, group: string, mclaren_group: string, product_id: string, cnt: bigint, product_name: string, special_craft_user: string]

In [16]:
from pyspark.sql.window import Window
Window_Spec  = Window.partitionBy("group", "mclaren_group").orderBy(col("craft_cnt").desc())

In [13]:
special_crafted_product = mclaren_user_special_craft.groupBy("group", "mclaren_group", "product_id", "product_name").agg(countDistinct("special_craft_user").alias("user_cnt"), sum("cnt").alias("craft_cnt"))

In [17]:
special_crafted_product_rank = special_crafted_product.withColumn("rank", row_number().over(Window_Spec)).orderBy("group", "mclaren_group", "rank").toPandas()

In [19]:
special_crafted_product_rank.to_csv("./special_crafted_product_rank.csv", index=False)

In [18]:
special_crafted_product_rank

,group,mclaren_group,product_id,product_name,user_cnt,craft_cnt,rank
0,M-1,N,currencydesc.chestkeypiece*1,None,201486,464517.0,1
1,M-1,N,currencydesc.chestkey*1,None,226217,368737.0,2
2,M-1,N,itemdesc.14300003,McLAREN LOOT CACHE,85242,85245.0,3
3,M-1,N,currencydesc.chestkey*3,None,34271,34272.0,4
4,M-1,N,itemdesc.12022024,McLaren Brake Calipers (McLaren Orange),5175,5175.0,5
5,M-1,N,itemdesc.12022023,McLaren Brake Calipers (Black),2713,2713.0,6
6,M-1,N,itemdesc.12022017,McLaren Wheels (Gloss Black),1531,1531.0,7
7,M-1,N,itemdesc.12022016,McLaren Elite Wheels (Gloss Black Diamond Cut),1432,1432.0,8
8,M-1,N,itemdesc.12022022,McLaren Elite Brake Calipers (Red),959,959.0,9
9,M-1,N,itemdesc.12022020,McLaren Elite Brake Calipers (Flux Green),516,516.0,10


In [57]:
crate_increased = load_schema.lobby(spark, "pc", "live", "WorkshopCrateIncreased", "2022-09-07", "2022-11-08")

In [59]:
event_loot_cache = crate_increased.where(col("ItemDescId") == "itemdesc.14300005")

In [61]:
event_loot_cache_by_user = event_loot_cache.groupBy("AccountId").agg(sum("Amount").alias("amount"))

In [62]:
event_loot_cache_by_user.select(max("amount")).show()

+-----------+
|max(amount)|
+-----------+
|         60|
+-----------+



In [23]:
# M-2, SB 유저 시나리오 

In [37]:
tmp_account_id = "account.00ead1dd557d4858a1e24ba43bdc4301"

In [52]:
tmp_account_id2 = "account.0183dcc0a6134aab93b598dc7c20d5c7"

In [25]:
df.where((col("group") == "M-2") & (col("mclaren_group") == "SB")).show(truncate=False)

+----------------------------------------+----------+--------+---------+--------+--------------+-------------+-----+-------------+-------------+--------+------------------+------------------+-----------------+--------------------+---------------------+---------+--------------------+------------+-------------+-------------------+------------------+---------------+-----------------------+---------------------------------+-----------------+------------+------------------+-------------------+---------------------+--------------------------------------+-------------------------------------+-------------------------+------------+
|account_id                              |pass_gcoin|paid_use|total_use|level_up|level_up_gcoin|product_gcoin|group|mclaren_group|play_date_cnt|play_cnt|play_min          |workshop_total_use|workshop_paid_use|non_mclaren_paid_use|non_mclaren_total_use|max_level|mclaren_own_date_num|pre_paid_use|pre_total_use|pre_gcoin_use_group|pre_workshop_gcoin|pre_if_workshop|pre

In [100]:
from datetime import datetime, timedelta

def load_data_mart(device, start_date, end_date, table_name="cash_mtx", join_with_meta=True):
    """
        load economy data_mart with its corresponding meta
    """
    from datetime import datetime, timedelta
    from pyspark.sql import Window
    from pyspark.sql import functions as F
    from pyspark.sql.types import StringType
    from pubg_util import mysql
    import boto3
    from pubg_util.loader.utils import validate_s3_path
    client = boto3.client('s3')
    bucket = {
        "cash_mtx" : "s3a://pubg-log-labs/data_mart/economy_v2/cash_mtx/{device}/{target_date}",
        "pu_master" : "s3a://pubg-log-labs/data_mart/economy_v2/pu_master/{device}/{target_date}",
        "gcoin_use" : "s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/{device}/{target_date}",
        "gcoin_topup" : "s3a://pubg-log-labs/data_mart/economy_v3/gcoin_topup/{device}/{target_date}",
        "gcoin_master" : "s3a://pubg-log-labs/data_mart/economy_v3/gcoin_master/{device}/{target_date}",
        "user_master" : "s3a://pubg-log-labs/data_mart/user_master_ver2/{target_date}/{device}"
    }
    meta_name = {
        "cash_mtx" : "meta_cash_mtx",
        "gcoin_use" : "meta_vc_sales_items"
    }
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    date_list = [(start_date + timedelta(d)).strftime("%Y-%m-%d") for d in range((end_date-start_date).days+1)]
    device_str = device.upper() if table_name=='user_master' else device
    path_list = [bucket[table_name].format(device=device_str, target_date=target_date) for target_date in date_list]
    # load_only valid paths
    valid_path_list = list(filter(lambda path: validate_s3_path(client, path), path_list))
    if len(valid_path_list) == 0:
        message = "no {} data in given period {}-{}".format(table_name, start_date, end_date)
        print(message)
        return None
    if device == 'pc' and table_name == 'cash_mtx':
        df_raw = spark.read.option("mergeSchema", "true").parquet(*valid_path_list)
    elif table_name == 'gcoin_use':
        df_raw = spark.read.option("mergeSchema", "true").parquet(*valid_path_list)
        df_raw_no_salesid = df_raw.where('date >= "2021-07-28" and sales_id is null').toPandas()
        if not df_raw_no_salesid.empty:
            message = 'GCOIN USE data exist: sales_id is null\n{}'.format(df_raw_no_salesid.to_string(index=False))
            print(message)
    else:
        df_raw = spark.read.parquet(*valid_path_list)
 
    missing_dates = list(set(path_list) - set(valid_path_list))
    if len(missing_dates) > 0:
        message = "no data exists in\n" + "\n".join(missing_dates)
        print(message)
    if join_with_meta is False or table_name not in ('cash_mtx', 'gcoin_use'):
        # if it's not cash_mtx or gcoin_use, return raw_df as there are no meta to join with
        return df_raw
    else:
        # load meta_cash_mtx
        meta_raw = (
            mysql.read_table(spark, 'metainfo', meta_name[table_name])
            .withColumnRenamed("platform", "platform_")
            .withColumnRenamed("product_id", "product_id_")
            .withColumnRenamed("price", "price_")
          )
        if table_name == 'gcoin_use':
            meta_raw = meta_raw.where("currency = 'gcoin'")
            df_raw = df_raw \
                .withColumn('is_salesid_exist',
                            F.when(F.col('sales_id').isNull(), F.lit(0)).otherwise(F.lit(1))) \
                .withColumn('sales_id_',
                            F.when(F.col('sales_id').startswith('salesitemdesc'),
                            F.split('sales_id', 'desc.')[1]).otherwise(F.col('sales_id'))) \
                .drop('sales_id')
        if device == 'console' and table_name == 'cash_mtx':
            # join condition
            condition = [
                df_raw.platform == meta_raw.platform_,
                df_raw.product_id == meta_raw.product_id_
            ]
            # join with meta
            # is_paid is hard coded for products that were provided to Stadia Pro users for free during promo period
            df = (
                df_raw
                .join(meta_raw, condition, 'left')
                .withColumn("transaction_id", lit(None).cast(StringType()))
                .withColumn("status", F.lit('Succeeded').cast(StringType()))
                .withColumnRenamed("time", "time_")
                .withColumn("time", F.coalesce(F.col("time_"), F.concat(F.col("date"), F.lit("T00:00:00"))))
                .drop('platform_', 'product_id_', 'time_')
                .withColumn('is_paid',
                    ~(
                        (F.col('product_id').isin(['PUBGPIONEREDITION', 'PUBGBDLDSLEISKIN']))
                    )
                ).where('is_paid')
            )
        else:
            meta_duplicate_count = (
                meta_raw
                .groupBy('product_id_', 'platform_')
                .agg(F.count(F.lit(1)).alias('duplications'))
            )
            window_spec = (
                Window
                .partitionBy('platform_', 'product_id_')
                .orderBy('start_time')
            )
            meta = (
                meta_raw.alias("meta_raw")
                .join(
                    meta_duplicate_count.alias("meta_duplicate_count"),
                    on=['product_id_', 'platform_'],
                    how='left')
                .select("meta_raw.*", "meta_duplicate_count.duplications")
                .withColumn(
                    "next_start_time",
                    F.coalesce(F.lead("start_time").over(window_spec), F.lit('2038-01-19 00:00:00'))
                )
            )
            if table_name == 'gcoin_use':
                condition = (
                (
                    (df_raw.is_salesid_exist == 1)
                    & (
                        (df_raw.platform == meta.platform_)
                        & (df_raw.sales_id_ == meta.sales_id)
                    )
                )
                | ( (df_raw.is_salesid_exist == 0)
                    & (df_raw.platform == meta.platform_)
                    & (df_raw.product_id == meta.product_id_)
                    & (
                        (
                            (df_raw.time >= meta.start_time)
                            & (df_raw.time < meta.next_start_time)
                            & (meta.duplications >= 2)
                        )
                            | (meta.duplications == 1)
                    )
                ))
                # join with meta
                df = (
                    df_raw
                    .join(meta, condition, 'left')
                    .drop('platform_', 'product_id_', 'next_start_time', "duplications")
                )
            else:
                condition = (
                    (df_raw.platform == meta.platform_)
                    & (df_raw.product_id == meta.product_id_)
                    & (
                        (
                            (df_raw.time >= meta.start_time)
                            & (df_raw.time < meta.next_start_time)
                            & (meta.duplications >= 2)
                        )
                            | (meta.duplications == 1)
                    )
                )
                # join with meta
                df = (
                    df_raw.drop('sales_id')
                    .join(meta, condition, 'left')
                    .drop('platform_', 'product_id_', 'next_start_time', "duplications")
                )
        if table_name == 'cash_mtx':
            df = df.withColumn("ingame_revenue", F.col("unit_sold") * F.col("ingame_price"))
 
        return df

In [58]:
start_date = "2022-09-07"
end_date = "2022-11-08"
mclaren_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name")=="202209_season_workshop")

KeyboardInterrupt: 

In [59]:
pre_start_date = "2022-07-13"
pre_end_date = "2022-09-06"
pre_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34821)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45227)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [60]:
event_name_by_type = pre_gcoin.select("event_type", "event_name").distinct().orderBy("event_type", "event_name").toPandas()

In [61]:
event_name_by_type

,event_type,event_name
0,collaboration,202207_balibandits_emote
1,collaboration,202208_assassinscreed
2,collaboration,202209_heymama_emote
3,collaboration,202210_deadbydaylight
4,collaboration,202210_momoland_emote
5,esports,202208_pgc
6,esports,202209_pcs7
7,esports,202209_pcs7_emote
8,esports,202210_pgc
9,esports,202210_pgc_emote


In [44]:
open_mclaren_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date).where(col("ItemDescId").isin(["itemdesc.14300003", "itemdesc.14300004", "itemdesc.14300005"]))

In [42]:
mclaren_gcoin.where(col("account_id") == tmp_account_id).orderBy("time").select("date", "product_name", "free_use", "paid_use", "qty").show(truncate=False)

+----------+-----------------------------+--------+--------+---+
|date      |product_name                 |free_use|paid_use|qty|
+----------+-----------------------------+--------+--------+---+
|2022-10-04|McLAREN LOOT CACHE PACK (x55)|460     |12040   |1  |
|2022-10-10|McLAREN LOOT CACHE PACK (x55)|4240    |20760   |2  |
+----------+-----------------------------+--------+--------+---+



In [51]:
open_mclaren_crate.where(col("AccountId") == tmp_account_id).orderBy("Time").select("date", "Time", "ItemDescId", "ItemsObtained", "OpenAmount").show(50, truncate=False)

+----------+----------------------------+-----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|date      |Time                        |ItemDescId       |ItemsObtained                                                                                                                                                                                                                                                                                                                     |OpenAmount|
+----------+----------------------------+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
mclaren_gcoin.where(col("account_id") == tmp_account_id2).orderBy("time").select("date", "product_name", "free_use", "paid_use", "qty").show(truncate=False)

+----------+-----------------------------+--------+--------+---+
|date      |product_name                 |free_use|paid_use|qty|
+----------+-----------------------------+--------+--------+---+
|2022-09-09|McLAREN LOOT CACHE PACK (x1) |0       |250     |1  |
|2022-09-10|McLAREN LOOT CACHE PACK (x55)|1500    |11000   |1  |
+----------+-----------------------------+--------+--------+---+



itemdesc.14300003	McLAREN LOOT CACHE  
itemdesc.14300004	McLAREN PRIME PARCEL  
itemdesc.14300005	McLAREN EVENT LOOT CACHE

itemdesc.12021002 - standard black  
itemdesc.12021003 - standard white  
itemdesc.12021001 - elite yellow

In [54]:
open_mclaren_crate.where(col("AccountId") == tmp_account_id2).orderBy("Time").select("date", "Time", "ItemDescId", "ItemsObtained", "OpenAmount").show(50, truncate=False)

+----------+----------------------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|date      |Time                        |ItemDescId       |ItemsObtained                                                                                                                                                                                                                                                                                                                            |OpenAmount|
+----------+----------------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58192)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

### SW, EY를 획득한 유저들의 제작 조합
select product_id, product_name, price 
from metainfo.meta_vc_sales_items
where platform = 'steam' and currency like '%seasonaltoken%'
order by price desc;

In [101]:
special_craft = special_craft.withColumn("type", \
    when(col("ItemId").isin(["itemdesc.12022004", "itemdesc.12022005", "itemdesc.12022006", "itemdesc.12022007", "itemdesc.12022008", "itemdesc.12022009", "itemdesc.12022010"]), lit("elite_paint")) \
    .when(col("ItemId").isin(["itemdesc.12022011", "itemdesc.12022012", "itemdesc.12022013", "itemdesc.12022014"]), lit("paint")) \
    .when(col("ItemId").isin(["itemdesc.12022016"]), lit("elite_wheels")) \
    .when(col("ItemId").isin(["itemdesc.12022017"]), lit("wheels")) \
    .when(col("ItemId").isin(["itemdesc.12022020", "itemdesc.12022021", "itemdesc.12022022"]), lit("elite_calipers")) \
    .when(col("ItemId").isin(["itemdesc.12022023", "itemdesc.12022024"]), lit("calipers")) \
    .otherwise(lit(None)))

In [102]:
package_craft = special_craft.withColumnRenamed("AccountId", "account_id").groupBy("account_id", "type").agg(count("*").alias("cnt"))

In [103]:
mclaren_owner = mysql.read_table(spark, "labs", "mclaren_owner")

In [104]:
sw_ey_owner = mclaren_owner.where(col("mclaren").isin(["standard_white", "elite_yellow"]))

In [105]:
both_owner = sw_ey_owner.groupBy("AccountId").agg(count("*").alias("cnt")).filter(col("cnt") == 2).select("AccountId")
both_owner = sw_ey_owner.join(both_owner, "AccountId").withColumn("mclaren", lit("both"))
sw_ey_owner = sw_ey_owner.join(both_owner.select("AccountId"), "AccountId", "leftanti")
sw_ey_owner = sw_ey_owner.unionByName(both_owner)

In [106]:
sw_ey_owner.select("mclaren").distinct().show(truncate=False)

+--------------+
|mclaren       |
+--------------+
|elite_yellow  |
|both          |
|standard_white|
+--------------+



In [107]:
package_craft = package_craft.join(sw_ey_owner.withColumnRenamed("AccountId", "account_id").select("account_id", "mclaren"), "account_id")

In [108]:
mix_match_tmp = package_craft.groupBy("mclaren", "account_id").agg(concat_ws(", ", sort_array(collect_set(col("type")))).alias("mix_match"))
mix_match = mix_match_tmp.groupBy("mclaren", "mix_match").agg(countDistinct("account_id").alias("user_cnt")).toPandas()

In [114]:
mix_match[mix_match["mclaren"] == "both"].sort_values(["mclaren", "user_cnt"], ascending=False)

,mclaren,mix_match,user_cnt
167,both,"calipers, elite_calipers, elite_paint, elite_w...",409
20,both,"calipers, elite_calipers, elite_paint, elite_w...",114
88,both,elite_paint,93
15,both,"elite_calipers, elite_paint, elite_wheels",57
162,both,,40
134,both,"calipers, elite_calipers, elite_paint, paint, ...",39
180,both,"elite_calipers, elite_paint, elite_wheels, paint",36
37,both,"calipers, elite_calipers, elite_paint, elite_w...",35
161,both,"calipers, elite_paint, paint, wheels",34
123,both,"elite_calipers, elite_paint, wheels",28


In [115]:
user_group = df.select("account_id", "group")
mix_match_by_group = mix_match_tmp.join(user_group, "account_id").groupBy("mclaren","group","mix_match").agg(countDistinct("account_id").alias("user_cnt")).toPandas()

In [118]:
mit_match2 = mix_match_tmp.join(user_group, "account_id").groupBy("mclaren", "mix_match").agg(countDistinct("account_id").alias("user_cnt")).toPandas()

In [116]:
mix_match.sort_values(["mclaren", "user_cnt"], ascending=False).to_csv("./mix_match.csv", index=False)

In [120]:
mit_match2.sort_values(["mclaren", "user_cnt"], ascending=False).to_csv("./mix_match2.csv", index=False)

In [121]:
mix_match.groupby("mclaren")["user_cnt"].sum()

mclaren
both                1232
elite_yellow       60817
standard_white    267346
Name: user_cnt, dtype: int64

In [122]:
mit_match2.groupby("mclaren")["user_cnt"].sum()

mclaren
both                1231
elite_yellow       60798
standard_white    267297
Name: user_cnt, dtype: int64

In [117]:
mix_match_by_group.sort_values(["mclaren", "group", "user_cnt"], ascending=False).to_csv("./mix_match_by_group.csv", index=False)

In [123]:
mix_match_by_group.groupby("mclaren")["user_cnt"].sum()

mclaren
both                1231
elite_yellow       60798
standard_white    267297
Name: user_cnt, dtype: int64

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 41046)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [ ]:
def get_workshop_rate(start_date, end_date):
    bp_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")) 
    
    workshop_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name")=="202207_workshop")
    mclaren_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name")=="202209_season_workshop")
    
    open_mclaren_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date).where(col("ItemDescId").isin(["itemdesc.14300003", "itemdesc.14300004", "itemdesc.14300005"]))
    open_normal_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date).where(~col("ItemDescId").isin(["itemdesc.14300003", "itemdesc.14300004", "itemdesc.14300005"]))
    
    mclaren_craft_items = mysql.read_table(spark, "labs", "item_meta_brodesc").where(col("item_name").like("%McLaren%")).withColumn('item_id_number', split(col("item_id"), '\.').getItem(1))
    craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date).withColumn('item_id_number', split(col("ItemDescId"), '\.').getItem(1))
    mclaren_craft = craft.join(mclaren_craft_items, "item_id_number")
    normal_craft = craft.join(mclaren_craft_items, "item_id_number", "leftanti")
    
    disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date)
    repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date)
    
    special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken")
    mclaren_special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency").like("%seasonaltoken%"))
    
    workshop_user = (bp_purchase.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("bp"))) \
        .unionByName(workshop_gcoin.select("account_id").distinct().withColumn("action", lit("workshop_gcoin"))) \
        .unionByName(mclaren_gcoin.select("account_id").distinct().withColumn("action", lit("mclaren_gcoin"))) \
        .unionByName(open_mclaren_crate.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("mclaren_crate_open"))) \
        .unionByName(open_normal_crate.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("normal_crate_open"))) \
        .unionByName(mclaren_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("mclaren_craft"))) \
        .unionByName(normal_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("normal_craft"))) \
        .unionByName(disassemble.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("disassemble"))) \
        .unionByName(repurpose.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("repurpose"))) \
        .unionByName(special_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("special_craft"))) \
        .unionByName(mclaren_special_craft.withColumnRenamed("AccountId", "account_id").select("account_id").distinct().withColumn("action", lit("mclaren_special_craft")))
    
    return workshop_user